# 개요
* GPT활용교육을 들은 내용을 별도로 정리
  * 1일차 : 네이버API 활용 : GPT를 활용한 변수 분석 등
  * 2일차 : beautifulsoup활용한 뉴스 및 증권사리포트 크롤링
  * 3일차 : 뉴스 및 증권사리포트 데이터 전처리 
  * 4일차 : 감성점수 변환
  * 5일차 : 백테스팅 실습

# 4일차 실습내용 정리


## (내용정리) OpenAI API를 활용한 감성 점수 추출: Few-shot Prompting 활용

* 감성 점수는 보통 합쳐서 1이 되게 한다 : 긍정(0.7), 중립(0.2), 부정(0.1)
* Few-shot Prompting : 특정 작업을 위해, 소량의 예제를 제공해 작업수행능력을 이끌어내는 법(기존 ML은 대량의 데이터 필요)
  * 장점 : 모델을 새로 학습시키지 않아도 작업 가능 / 작은 데이터로도 가능 / 유연성(다양한 작업에 활용 가능)
* Few-shot Prompting은 프롬프트에 예제 데이터를 포함하여 적용
  * 프롬프트 템플릿 (3번이 Few-shot prompting)
    ```
    1. 시스템 역할 설명
    당신은 주식과 관련된 뉴스 또는 증권사 리포트에서 감성 점수를 추출하는 전문가입니다.
    주어진 텍스트를 분석하여 감성 점수를 다음의 기준에 따라 평가하세요.

    2. 감성 점수 기준 정의
    긍정 (Positive): 해당 텍스트가 주식의 긍정적인 영향을 암시하면 점수는 1에 가까움.
    중립 (Neutral): 해당 텍스트가 중립적이거나 영향을 판단하기 어려운 경우 점수는 0.5에 가까움.
    부정 (Negative): 해당 텍스트가 주식에 부정적인 영향을 암시하면 점수는 0에 가까움.
    각 긍정, 중립, 부정 점수의 합이 1이 되게 해주세요.

    3. 입력 데이터 예시 제공
    "다음은 입력 데이터와 기대하는 출력 데이터의 예시입니다."
    예시 1:
    입력: "해당 회사의 매출이 25% 증가하며 시장 기대치를 초과했습니다."
    출력: 긍정 0.8 중립 0.1 부정 0.1
    예시 2:
    입력: "글로벌 경제 둔화로 인해 매출 감소가 예상됩니다."
    출력: 긍정 0.1 중립 0.2 부정 0.7
    예시 3:
    입력: "회사는 안정적인 성장을 유지하고 있으며, 큰 변화가 없습니다."
    출력: 긍정 0.1 중립 0.8 부정 0.1

    4. 지시사항 추가
    "아래에 주어진 텍스트를 분석하고, 감성 점수를 출력 형식에 맞춰 응답하세요."
    ```

## 실습 : 기존 내용 추출하여 OpenaiAPI로 감성분석

In [ ]:
# 요약된 기사 데이터 불러오기

import pandas as pd

summary_news = pd.read_csv('3_2_데이터/summary_news.csv')
summary_report = pd.read_csv('3_2_데이터/summary_report.csv')

In [ ]:
summary_news.head()

,Unnamed: 0.1,Unnamed: 0,company,title,date,summary,generated_summary,positive_score,negative_score,neutral_score,ticker
0,0,0,BGF,"[날개 단 유통 재벌家 아들]④BGF, 10분의1 토막 주가에 한살 터울 형제 승계...",2023-01-31,홍정국 BGF리테일 대표(왼쪽)와 차남 홍정혁 BGF에코머티리얼즈 대표/사진= BG...,"BGF그룹의 2세 승계가 진행 중이며, 장남 홍정국이 편의점 CU를 운영하는 BGF...",0.30,0.50,0.20,027410.KS
1,1,1,BGF,"""농가 상생에 진심"" CU, 진도 대파 간편식 시리즈 출시",2023-01-31,BGF리테일(282330)이 운영하는 편의점 CU가 지역 농가 상생 프로젝트로 전라...,CU가 진도 지역 농가와의 상생을 위해 대파를 활용한 간편식 시리즈를 출시했다. 이...,0.75,0.05,0.20,027410.KS
2,2,2,BGF,"BGF리테일, 협력사와 경영 특강 및 ESG 경영 선언식 진행",2023-01-30,급변하는 트렌드를 읽고 시장 경쟁력을 강화하는 방법 주제로 협력사 동반성장 특강 진...,BGF리테일은 26일 서울 강남구 BGF사옥에서 전국 80여 개 협력사 임직원과 함...,0.75,0.05,0.20,027410.KS
3,3,3,BGF,[브리프] LG생활건강 GS25 세븐일레븐 BGF 그린벳,2023-01-30,"CU, 하이포크 삼겹살 특가 판매 CU 하이포크 삼겹살 할인ㅣBGF CU가 편의점 ...",CU는 하이포크 삼겹살을 특가로 판매하며 신선육 운영을 전국으로 확대한다고 발표했다...,0.70,0.05,0.25,027410.KS
4,4,4,BGF,"[게시판] BGF리테일, 협력사와 경영특강·ESG 선언식",2023-01-29,▲ CU를 운영하는 BGF리테일은 지난 26일 서울 강남구 사옥에서 협력사 대상 경...,BGF리테일은 26일 서울에서 협력사를 대상으로 경영 특강과 ESG 경영 선언식을 ...,0.70,0.10,0.20,027410.KS


In [ ]:
summary_report.head()

,Unnamed: 0.1,Unnamed: 0,company,title,securities_company,attachment,date,summarize_text,generated_summary,positive_score,negative_score,neutral_score,ticker
0,0,0,LG에너지솔루션,2023년 판매 고객사에 따른 실적 차별화,메리츠증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,2023. 1. 30 \nLG에너지솔루션 373220 \n2023년 판매 고객사...,"LG에너지솔루션(373220)은 2022년 매출 25.5조원, 영업이익 1.2조원으...",0.70,0.15,0.15,373220.KS
1,1,1,삼성SDI,달라지는 업황에 따른 기대감,DS투자증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,"\nBUY (유지) \n \n목표주가 (상향) 890,000원 \n현재주가 (0...",**요약: 삼성SDI 리포트**\n\n- **투자등급:** BUY (유지)\n- *...,0.65,0.15,0.20,006400.KS
2,2,3,삼성SDI,4Q22 Review: 대체적으로 컨센 부합,교보증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n Compan y Report \n삼성SDI \n006400 \n ...,**삼성SDI 2023년 1분기 증권사 리포트 요약**\n\n- **투자의견**: ...,0.75,0.10,0.15,006400.KS
3,3,4,삼성중공업,4Q22 Review: 동트기 전 마지막 성장통,교보증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n Company Report \n삼성중공업 \n010140 \n Ja...,**삼성중공업 회사 리포트 요약 (2023년 1월 31일)**\n\n- **투자의견...,0.65,0.25,0.10,010140.KS
4,4,5,SGC에너지,상한제 규제 리스크 해소 여부에 관심,하나증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n2023년 1월 31일 I Equity Research \nSGC에너지...,"2023년 1월 31일 발표된 SGC에너지(005090) 증권사 리포트에 따르면, ...",0.70,0.10,0.20,005090.KS


In [ ]:
# 강의 제공파일이 결과컬럼까지 포함된 버전으로 잘못제공되어 필요컬럼만 가져옴

In [ ]:
summary_news_origin = summary_news[['company', 'title', 'date', 'summary',
       'generated_summary', 'ticker']].copy()
summary_report_origin = summary_report[['company', 'title', 'securities_company',
       'attachment', 'date', 'summarize_text', 'generated_summary',
       'ticker']].copy()

In [ ]:
# pandas apply에 활용할 OpenAI로 요약해주는 함수 작성

from openai import OpenAI
import json

# OpenAI API key
with open("keys.json", "r") as f:
    data = json.load(f)

open_ai_key = data.get("open_ai_key")

# 함수 작성
def get_sentiment_score(row):
    prompt = f"""
            당신은 주식과 관련된 뉴스 또는 증권사 리포트에서 감성 점수를 추출하는 전문가입니다.
            주어진 텍스트를 분석하여 감성 점수를 다음의 기준에 따라 평가하세요.

            - 긍정 (Positive): 해당 텍스트가 주식의 긍정적인 영향을 암시하는 경우
            - 중립 (Neutral): 해당 텍스트가 중립적이거나 영향을 판단하기 어려운 경우
            - 부정 (Negative): 해당 텍스트가 주식에 부정적인 영향을 암시하는 경우
            각 긍정, 중립, 부정 점수의 합이 1이 되게 해주세요.

            다음은 입력 데이터와 기대하는 출력 데이터의 예시입니다.
            예시 1:
            입력: 해당 회사의 매출이 25% 증가하며 시장 기대치를 초과했습니다.
            출력: 긍정 0.83 중립 0.13 부정 0.04
            예시 2:
            입력: 글로벌 경제 둔화로 인해 매출 감소가 예상됩니다.
            출력: 긍정 0.03 중립 0.23 부정 0.74
            예시 3:
            입력: 회사는 안정적인 성장을 유지하고 있으며, 큰 변화가 없습니다.
            출력: 긍정 0.13 중립 0.83 부정 0.04

            아래에 주어진 텍스트를 분석하고, 감성 점수를 출력 형식에 맞춰 응답하세요.

            {row['generated_summary']}
            """


    client = OpenAI(api_key =open_ai_key
                    )
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user","content": prompt}]
    )

    # print(completion.choices[0].message.content)

    results = completion.choices[0].message.content
    positive = float(results.split("긍정 ")[1].split(' ')[0].strip())
    neutral = float(results.split("중립 ")[1].split(' ')[0].strip())
    negative = float(results.split("부정 ")[1].split(' ')[0].strip())
    # print(f"{positive} {neutral} {negative}")

    return positive, neutral, negative

In [ ]:
# API 토큰 절약을 위해 10개 행만 추림

summary_news_origin_row10 = summary_news_origin[:10].copy()
summary_report_origin_row10 = summary_report_origin[:10].copy()

In [ ]:
summary_news_origin_row10

,company,title,date,summary,generated_summary,ticker
0,BGF,"[날개 단 유통 재벌家 아들]④BGF, 10분의1 토막 주가에 한살 터울 형제 승계...",2023-01-31,홍정국 BGF리테일 대표(왼쪽)와 차남 홍정혁 BGF에코머티리얼즈 대표/사진= BG...,"BGF그룹의 2세 승계가 진행 중이며, 장남 홍정국이 편의점 CU를 운영하는 BGF...",027410.KS
1,BGF,"""농가 상생에 진심"" CU, 진도 대파 간편식 시리즈 출시",2023-01-31,BGF리테일(282330)이 운영하는 편의점 CU가 지역 농가 상생 프로젝트로 전라...,CU가 진도 지역 농가와의 상생을 위해 대파를 활용한 간편식 시리즈를 출시했다. 이...,027410.KS
2,BGF,"BGF리테일, 협력사와 경영 특강 및 ESG 경영 선언식 진행",2023-01-30,급변하는 트렌드를 읽고 시장 경쟁력을 강화하는 방법 주제로 협력사 동반성장 특강 진...,BGF리테일은 26일 서울 강남구 BGF사옥에서 전국 80여 개 협력사 임직원과 함...,027410.KS
3,BGF,[브리프] LG생활건강 GS25 세븐일레븐 BGF 그린벳,2023-01-30,"CU, 하이포크 삼겹살 특가 판매 CU 하이포크 삼겹살 할인ㅣBGF CU가 편의점 ...",CU는 하이포크 삼겹살을 특가로 판매하며 신선육 운영을 전국으로 확대한다고 발표했다...,027410.KS
4,BGF,"[게시판] BGF리테일, 협력사와 경영특강·ESG 선언식",2023-01-29,▲ CU를 운영하는 BGF리테일은 지난 26일 서울 강남구 사옥에서 협력사 대상 경...,BGF리테일은 26일 서울에서 협력사를 대상으로 경영 특강과 ESG 경영 선언식을 ...,027410.KS
5,BGF,"""신선육 파는 편의점""...CU, 삼겹살 특가 판매",2023-01-30,사진제공=BGF리테일 CU는 근거리 소비 트렌드에 맞춰 신선육 판매를 전국 매장으로...,"CU는 근거리 소비 트렌드에 맞춰 신선육 판매를 전국 매장으로 확대하며, 삼겹살 초...",027410.KS
6,BGF,"[실적랭킹] BGF리테일, 지난해 4Q 예상 매출액 증가율 1위 할인∙편의점주",2023-01-31,한국 주식 시장의 할인점∙편의점∙홈쇼핑주 가운데 지난해 4분기 예상 매출액 증가율 ...,"BGF리테일이 지난해 4분기 예상 매출액 증가율 1위를 기록하며, 매출액은 1조93...",027410.KS
7,BGF,"전남도, 2022년산 새청무쌀 1만 3000t BGF리테일 공급",2023-01-25,있는 BGF리테일에 2022년산 새청무쌀 1만 3000여t을 공급한다고 25일 밝혔...,전남도가 2022년산 새청무쌀 1만 3000t을 BGF리테일에 공급한다고 발표했다....,027410.KS
8,BGF,"[편의점에서는] CU, 진도 대파 간편식 시리즈 출시 外 세븐일레븐, GS25",2023-01-31,(제공: BGF리테일) 31일 편의점 소식을 전한다. ◆“농가 상생에 진심”… CU...,CU가 진도 대파를 활용한 간편식 시리즈를 출시하며 지역 농가와의 상생을 도모하고 ...,027410.KS
9,BGF,"""1400원으로 한 끼 해결""…'짠물 소비족'에 인기 폭발 [박종관의 유통관통]",2023-01-27,27일 BGF리테일이 운영하는 편의점 CU에 따르면 지난해 10~12월 대용량 삼각...,CU 편의점의 대용량 삼각김밥 매출이 지난해 10~12월 동안 전년 대비 35.2%...,027410.KS


In [ ]:
summary_report_origin_row10

,company,title,securities_company,attachment,date,summarize_text,generated_summary,ticker
0,LG에너지솔루션,2023년 판매 고객사에 따른 실적 차별화,메리츠증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,2023. 1. 30 \nLG에너지솔루션 373220 \n2023년 판매 고객사...,"LG에너지솔루션(373220)은 2022년 매출 25.5조원, 영업이익 1.2조원으...",373220.KS
1,삼성SDI,달라지는 업황에 따른 기대감,DS투자증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,"\nBUY (유지) \n \n목표주가 (상향) 890,000원 \n현재주가 (0...",**요약: 삼성SDI 리포트**\n\n- **투자등급:** BUY (유지)\n- *...,006400.KS
2,삼성SDI,4Q22 Review: 대체적으로 컨센 부합,교보증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n Compan y Report \n삼성SDI \n006400 \n ...,**삼성SDI 2023년 1분기 증권사 리포트 요약**\n\n- **투자의견**: ...,006400.KS
3,삼성중공업,4Q22 Review: 동트기 전 마지막 성장통,교보증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n Company Report \n삼성중공업 \n010140 \n Ja...,**삼성중공업 회사 리포트 요약 (2023년 1월 31일)**\n\n- **투자의견...,010140.KS
4,SGC에너지,상한제 규제 리스크 해소 여부에 관심,하나증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n2023년 1월 31일 I Equity Research \nSGC에너지...,"2023년 1월 31일 발표된 SGC에너지(005090) 증권사 리포트에 따르면, ...",005090.KS
5,한전KPS,중장기 실적은 우상향 추세,하나증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n2023년 1월 31일 I Equity Research \n한전KPS (...,**한전KPS (051600) 투자 요약**\n\n- **투자의견:** 매수 (BU...,051600.KS
6,삼성중공업,2023년은 흑자전환의 원년,하나증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n2023년 1월 31일 I Equity Research \n삼성중공업 ...,삼성중공업(010140)에 대한 2023년 증권사 리포트는 긍정적인 전망을 제시하고...,010140.KS
7,삼성SDI,투자 증가세 뚜렷,하나증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n2023년 1월 31일 I Equity Research \n삼성SDI ...,**삼성SDI (006400) 투자 리포트 요약**\n\n- **투자 의견**: B...,006400.KS
8,GS건설,올해 1분기 마진 확인이 필요,하나증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n2023년 1월 31일 I Equity Research \nGS건설 (...,"GS건설에 대한 2023년 1분기 분석 보고서에서, 투자의견은 중립(Neutral)...",006360.KS
9,삼성SDI,성장에 대한 의구심 불식,한화투자증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,2023년 01월 31일 \n기업분석 \n \n \n \n삼성SDI (006400...,**삼성SDI 기업 분석 요약 (2023년 1월 31일)**\n\n삼성SDI는 4분...,006400.KS


In [ ]:
# 뉴스 감성분석
summary_news_origin_row10[['positive_score', 'neutral_score', 'negative_score']] = summary_news_origin_row10.apply(lambda row: pd.Series(get_sentiment_score(row)), axis=1)
summary_news_origin_row10.to_excel('3_2_데이터/summary_news_result.xlsx')
summary_news_origin_row10

,company,title,date,summary,generated_summary,ticker,positive_score,neutral_score,negative_score
0,BGF,"[날개 단 유통 재벌家 아들]④BGF, 10분의1 토막 주가에 한살 터울 형제 승계...",2023-01-31,홍정국 BGF리테일 대표(왼쪽)와 차남 홍정혁 BGF에코머티리얼즈 대표/사진= BG...,"BGF그룹의 2세 승계가 진행 중이며, 장남 홍정국이 편의점 CU를 운영하는 BGF...",027410.KS,0.10,0.25,0.65
1,BGF,"""농가 상생에 진심"" CU, 진도 대파 간편식 시리즈 출시",2023-01-31,BGF리테일(282330)이 운영하는 편의점 CU가 지역 농가 상생 프로젝트로 전라...,CU가 진도 지역 농가와의 상생을 위해 대파를 활용한 간편식 시리즈를 출시했다. 이...,027410.KS,0.75,0.20,0.05
2,BGF,"BGF리테일, 협력사와 경영 특강 및 ESG 경영 선언식 진행",2023-01-30,급변하는 트렌드를 읽고 시장 경쟁력을 강화하는 방법 주제로 협력사 동반성장 특강 진...,BGF리테일은 26일 서울 강남구 BGF사옥에서 전국 80여 개 협력사 임직원과 함...,027410.KS,0.77,0.18,0.05
3,BGF,[브리프] LG생활건강 GS25 세븐일레븐 BGF 그린벳,2023-01-30,"CU, 하이포크 삼겹살 특가 판매 CU 하이포크 삼겹살 할인ㅣBGF CU가 편의점 ...",CU는 하이포크 삼겹살을 특가로 판매하며 신선육 운영을 전국으로 확대한다고 발표했다...,027410.KS,0.78,0.15,0.07
4,BGF,"[게시판] BGF리테일, 협력사와 경영특강·ESG 선언식",2023-01-29,▲ CU를 운영하는 BGF리테일은 지난 26일 서울 강남구 사옥에서 협력사 대상 경...,BGF리테일은 26일 서울에서 협력사를 대상으로 경영 특강과 ESG 경영 선언식을 ...,027410.KS,0.78,0.17,0.05
5,BGF,"""신선육 파는 편의점""...CU, 삼겹살 특가 판매",2023-01-30,사진제공=BGF리테일 CU는 근거리 소비 트렌드에 맞춰 신선육 판매를 전국 매장으로...,"CU는 근거리 소비 트렌드에 맞춰 신선육 판매를 전국 매장으로 확대하며, 삼겹살 초...",027410.KS,0.75,0.20,0.05
6,BGF,"[실적랭킹] BGF리테일, 지난해 4Q 예상 매출액 증가율 1위 할인∙편의점주",2023-01-31,한국 주식 시장의 할인점∙편의점∙홈쇼핑주 가운데 지난해 4분기 예상 매출액 증가율 ...,"BGF리테일이 지난해 4분기 예상 매출액 증가율 1위를 기록하며, 매출액은 1조93...",027410.KS,0.88,0.10,0.02
7,BGF,"전남도, 2022년산 새청무쌀 1만 3000t BGF리테일 공급",2023-01-25,있는 BGF리테일에 2022년산 새청무쌀 1만 3000여t을 공급한다고 25일 밝혔...,전남도가 2022년산 새청무쌀 1만 3000t을 BGF리테일에 공급한다고 발표했다....,027410.KS,0.82,0.13,0.05
8,BGF,"[편의점에서는] CU, 진도 대파 간편식 시리즈 출시 外 세븐일레븐, GS25",2023-01-31,(제공: BGF리테일) 31일 편의점 소식을 전한다. ◆“농가 상생에 진심”… CU...,CU가 진도 대파를 활용한 간편식 시리즈를 출시하며 지역 농가와의 상생을 도모하고 ...,027410.KS,0.76,0.20,0.04
9,BGF,"""1400원으로 한 끼 해결""…'짠물 소비족'에 인기 폭발 [박종관의 유통관통]",2023-01-27,27일 BGF리테일이 운영하는 편의점 CU에 따르면 지난해 10~12월 대용량 삼각...,CU 편의점의 대용량 삼각김밥 매출이 지난해 10~12월 동안 전년 대비 35.2%...,027410.KS,0.74,0.18,0.08


In [ ]:
# 증권 리포트 감성분석
summary_report_origin_row10[['positive_score', 'neutral_score', 'negative_score']] = summary_report_origin_row10.apply(lambda row: pd.Series(get_sentiment_score(row)), axis=1)
summary_report_origin_row10.to_excel('3_2_데이터/summary_report_result.xlsx')
summary_report_origin_row10

,company,title,securities_company,attachment,date,summarize_text,generated_summary,ticker,positive_score,neutral_score,negative_score
0,LG에너지솔루션,2023년 판매 고객사에 따른 실적 차별화,메리츠증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,2023. 1. 30 \nLG에너지솔루션 373220 \n2023년 판매 고객사...,"LG에너지솔루션(373220)은 2022년 매출 25.5조원, 영업이익 1.2조원으...",373220.KS,0.76,0.18,0.06
1,삼성SDI,달라지는 업황에 따른 기대감,DS투자증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,"\nBUY (유지) \n \n목표주가 (상향) 890,000원 \n현재주가 (0...",**요약: 삼성SDI 리포트**\n\n- **투자등급:** BUY (유지)\n- *...,006400.KS,0.70,0.20,0.10
2,삼성SDI,4Q22 Review: 대체적으로 컨센 부합,교보증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n Compan y Report \n삼성SDI \n006400 \n ...,**삼성SDI 2023년 1분기 증권사 리포트 요약**\n\n- **투자의견**: ...,006400.KS,0.79,0.15,0.06
3,삼성중공업,4Q22 Review: 동트기 전 마지막 성장통,교보증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n Company Report \n삼성중공업 \n010140 \n Ja...,**삼성중공업 회사 리포트 요약 (2023년 1월 31일)**\n\n- **투자의견...,010140.KS,0.67,0.25,0.08
4,SGC에너지,상한제 규제 리스크 해소 여부에 관심,하나증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n2023년 1월 31일 I Equity Research \nSGC에너지...,"2023년 1월 31일 발표된 SGC에너지(005090) 증권사 리포트에 따르면, ...",005090.KS,0.75,0.20,0.05
5,한전KPS,중장기 실적은 우상향 추세,하나증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n2023년 1월 31일 I Equity Research \n한전KPS (...,**한전KPS (051600) 투자 요약**\n\n- **투자의견:** 매수 (BU...,051600.KS,0.75,0.15,0.10
6,삼성중공업,2023년은 흑자전환의 원년,하나증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n2023년 1월 31일 I Equity Research \n삼성중공업 ...,삼성중공업(010140)에 대한 2023년 증권사 리포트는 긍정적인 전망을 제시하고...,010140.KS,0.75,0.15,0.10
7,삼성SDI,투자 증가세 뚜렷,하나증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n2023년 1월 31일 I Equity Research \n삼성SDI ...,**삼성SDI (006400) 투자 리포트 요약**\n\n- **투자 의견**: B...,006400.KS,0.75,0.20,0.05
8,GS건설,올해 1분기 마진 확인이 필요,하나증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n2023년 1월 31일 I Equity Research \nGS건설 (...,"GS건설에 대한 2023년 1분기 분석 보고서에서, 투자의견은 중립(Neutral)...",006360.KS,0.20,0.64,0.16
9,삼성SDI,성장에 대한 의구심 불식,한화투자증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,2023년 01월 31일 \n기업분석 \n \n \n \n삼성SDI (006400...,**삼성SDI 기업 분석 요약 (2023년 1월 31일)**\n\n삼성SDI는 4분...,006400.KS,0.65,0.25,0.10


## 처음 제공된 기초 코드 샘플

In [ ]:
import pandas as pd

summary_news = pd.read_csv('3_2_데이터/summary_news.csv')
summary_report = pd.read_csv('3_2_데이터/summary_report.csv')

summary_news.head()
summary_report.head()

In [ ]:
from openai import OpenAI
def get_sentiment_score(row):
    prompt = f"""
            당신은 주식과 관련된 뉴스 또는 증권사 리포트에서 감성 점수를 추출하는 전문가입니다.
            주어진 텍스트를 분석하여 감성 점수를 다음의 기준에 따라 평가하세요.

            - 긍정 (Positive): 해당 텍스트가 주식의 긍정적인 영향을 암시하는 경우
            - 중립 (Neutral): 해당 텍스트가 중립적이거나 영향을 판단하기 어려운 경우
            - 부정 (Negative): 해당 텍스트가 주식에 부정적인 영향을 암시하는 경우
            각 긍정, 중립, 부정 점수의 합이 1이 되게 해주세요.

            다음은 입력 데이터와 기대하는 출력 데이터의 예시입니다.
            예시 1:
            입력: 해당 회사의 매출이 25% 증가하며 시장 기대치를 초과했습니다.
            출력: 긍정 0.83 중립 0.13 부정 0.04
            예시 2:
            입력: 글로벌 경제 둔화로 인해 매출 감소가 예상됩니다.
            출력: 긍정 0.03 중립 0.23 부정 0.74
            예시 3:
            입력: 회사는 안정적인 성장을 유지하고 있으며, 큰 변화가 없습니다.
            출력: 긍정 0.13 중립 0.83 부정 0.04

            아래에 주어진 텍스트를 분석하고, 감성 점수를 출력 형식에 맞춰 응답하세요.

            {row['generated_summary']}
            """


    client = OpenAI(api_key ="")
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user","content": prompt}]
    )

    print(completion.choices[0].message.content)

    results = completion.choices[0].message.content
    positive = float(results.split("긍정 ")[1].split(' ')[0].strip())
    neutral = float(results.split("중립 ")[1].split(' ')[0].strip())
    negative = float(results.split("부정 ")[1].split(' ')[0].strip())
    print(f"{positive} {neutral} {negative}")

    return positive, neutral, negative



In [ ]:
summary_news[['positive_score', 'neutral_score', 'negative_score']] = summary_news.apply(lambda row: pd.Series(get_sentiment_score(row)), axis=1)

In [ ]:
summary_report[['positive_score', 'neutral_score', 'negative_score']] = summary_report.apply(lambda row: pd.Series(get_sentiment_score(row)), axis=1)